In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

tqdm.pandas(desc='progress')

In [ ]:
# fonte: https://www.kaggle.com/datasets/sarvaninandipati/brazils-medical-appointment-show-or-no-show
df = pd.read_excel('appointment_prediction_data.xlsx')

# fonte: Vitória (Censo e UDH) - Atlas do Desenvolvimento Humano no Brasil - Ipea (https://dados.gov.br/dados/conjuntos-dados/atlasbrasil)
aux = pd.read_csv('RM_63200_Vitória_Base_UDH_2000_2010.csv')

In [ ]:
def uppercase(x):
    return str(x).upper()

aux['NOME_UDH'] = aux['NOME_UDH'].progress_apply(lambda x: uppercase(x))
dados_renda_per_capta = aux[['NOME_UDH', 'RDPC']]
dados_renda_per_capta = dados_renda_per_capta.groupby('NOME_UDH').mean().reset_index()

In [ ]:
# fonte: Plano Municipal de Saúde 2022-2025 Vitória - 2022 (https://m.vitoria.es.gov.br/download.php?tipo=1&id=3521)
regioes_saude = {
    'CONTINENTAL': ['AEROPORTO','ANTÔNIO HONÓRIO','BOA VISTA','JABOUR','JARDIM CAMBURI',
                    'JARDIM DA PENHA','MARIA ORTIZ','MATA DA PRAIA','MORADA DE CAMBURI',
                    'PONTAL DE CAMBURI','REPÚBLICA','SEGURANÇA DO LAR','SOLON BORGES',
                    'GOIABEIRAS', 'PARQUE INDUSTRIAL'],
    'CENTRO': ['CENTRO','DO CABRAL','DO MOSCOSO','DO QUADRO','FONTE GRANDE',
                     'ILHA DO PRÍNCIPE','PARQUE MOSCOSO','PIEDADE','SANTA CLARA',
                     'SANTA TEREZA','VILA RUBIM'],
    'SANTO ANTÔNIO': ['ARIOVALDO FAVALESSA','BELA VISTA','CARATOÍRA','ESTRELINHA',
                      'GRANDE VITÓRIA','INHANGUETÁ','MÁRIO CYPRESTE','SANTO ANTÔNIO',
                      'UNIVERSITÁRIO'],
    'MARUÍPE': ['ANDORINHAS','BONFIM','CONSOLAÇÃO','DA PENHA','DE LOURDES','GURIGICA',
                'HORTO','ITARARÉ','JOANA D\'ARC','MARUÍPE','SANTA CECÍLIA','SANTA MARTHA',
                'SANTOS DUMONT','SÃO BENEDITO','SÃO CRISTÓVÃO','TABUAZEIRO'],
    'SÃO PEDRO': ['COMDUSA','CONQUISTA','ILHA DAS CAIEIRAS','NOVA PALESTINA','REDENÇÃO',
                  'RESISTÊNCIA','SANTO ANDRÉ','SANTOS REIS','SÃO JOSÉ','SÃO PEDRO'],
    'FORTE SÃO JOÃO': ['BARRO VERMELHO','BENTO FERREIRA','CRUZAMENTO','ENSEADA DO SUÁ',
                       'FORTE SÃO JOÃO','FRADINHOS','MONTE BELO','ILHA DE SANTA MARIA',
                       'ILHA DO BOI','ILHA DO FRADE','JESUS DE NAZARETH','JUCUTUQUARA',
                       'NAZARETH','PRAIA DO CANTO','PRAIA DO SUÁ','ROMÃO','SANTA HELENA',
                       'SANTA LÚCIA','SANTA LUÍZA', 'ILHAS OCEÂNICAS DE TRINDADE']
}

def regiao_saude(bairro):
    for regiao, bairros in regioes_saude.items():
        if bairro in bairros:
            return regiao

def get_renda(bairro):
    renda = []
    for idx, row in dados_renda_per_capta.iterrows():
        if bairro in row['NOME_UDH']:
            renda.append(row['RDPC'])
    if not renda:
        return 0
    return sum(renda)/len(renda)

In [ ]:
# remover linhas duplicadas
df.drop_duplicates(subset='AppointmentID', keep='first')

# alterar coluna "No-show" para algo mais intuitivo
df['No-show'].replace({'Yes': 0, 'No': 1}, inplace=True)
df.rename(columns={'No-show': 'Compareceu'}, inplace=True)

# corrigir gêneros
df['Gender'].replace({'Female':'F','Male':'M'}, inplace=True)

# corrigir bairros
df['Neighbourhood'].replace({'RESIST√äNCIA':'RESISTÊNCIA','REP√öBLICA':'REPÚBLICA','S√ÉO PEDRO':'SÃO PEDRO',
                             'S√ÉO CRIST√ìV√ÉO':'SÃO CRISTÓVÃO','MARU√çPE':'MARUÍPE','GRANDE VIT√ìRIA':'GRANDE VITÓRIA',
                             'S√ÉO BENEDITO':'SÃO BENEDITO','SANTO ANDR√â':'SANTO ANDRÉ','ANT√îNIO HON√ìRIO':'ANTÔNIO HONÓRIO',
                             'M√ÅRIO CYPRESTE':'MÁRIO CYPRESTE','SANTO ANT√îNIO':'SANTO ANTÔNIO','PRAIA DO SU√Å':'PRAIA DO SUÁ',
                             'ITARAR√â':'ITARARÉ','INHANGUET√Å':'INHANGUETÁ','UNIVERSIT√ÅRIO':'UNIVERSITÁRIO',
                             'S√ÉO JOS√â':'SÃO JOSÉ','REDEN√á√ÉO':'REDENÇÃO','CARATO√çRA':'CARATOÍRA',
                             'ILHA DO PR√çNCIPE': 'ILHA DO PRÍNCIPE','ROM√ÉO': 'ROMÃO', 'FORTE S√ÉO JO√ÉO': 'FORTE SÃO JOÃO',
                             'JOANA D¬¥ARC': 'JOANA D\'ARC','CONSOLA√á√ÉO': 'CONSOLAÇÃO', 'SANTA CEC√çLIA': 'SANTA CECÍLIA',
                             'SANTA L√öCIA': 'SANTA LÚCIA','SANTA LU√çZA': 'SANTA LUÍZA', 'SEGURAN√áA DO LAR': 'SEGURANÇA DO LAR',
                             'Tabuazerio': 'TABUAZEIRO','ILHAS OCE√ÇNICAS DE TRINDADE': 'ILHAS OCEÂNICAS DE TRINDADE',
                             'ENSEADA DO SU√Å': 'ENSEADA DO SUÁ', 'DE LOURDE': 'DE LOURDES'}, inplace=True)
df['Neighbourhood'] = df.Neighbourhood.str.upper()

# adicionar macro-regiões de saúde
df['Regiao_Saude'] = df['Neighbourhood'].progress_apply(lambda x: regiao_saude(x))

# adicionar tempo entre 'dia que marcou a consulta' e 'dia da consulta'
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay']).dt.date
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay']).dt.date
df['time_delta'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days

# adicionar renda média por bairro
df['Renda'] = df['Neighbourhood'].progress_apply(lambda x: get_renda(x))

In [ ]:
not_emergency = df.query('time_delta > 0')

In [ ]:
not_emergency.groupby(['Neighbourhood', 'Renda'])['Compareceu'].mean().sort_values(ascending=True)*100

In [ ]:
not_emergency.groupby('Regiao_Saude')['Renda'].mean().sort_values(ascending=True)

In [ ]:
b_r_p = not_emergency.groupby(['Neighbourhood', 'Renda'])['Compareceu'].mean().sort_values(ascending=True)*100
x = [b[0] for b in b_r_p.index]
y = [b[1] for b in b_r_p.index]
plt.figure(figsize=(20,2))
plt.subplot(151)
plt.plot(x, b_r_p.values)
plt.subplot(152)
plt.plot(x, y, )
plt.show()